In [1]:
import MisrToolkit as Mtk 	# python2.7
import os, glob, cv2, gdal
import numpy as np
# import PIL
from PIL import Image

## plot f()

In [2]:
def arr2img_func(in_arr_2d, img_label, img_dir):

    from matplotlib import image as pltimg, pyplot as plt  #  pyplot uses the actual RGB values as they are, more accurate than PIL
    
    save_mode = True
    print('-> save mode: %s' % save_mode)
        
#     if (in_arr_2d.max() < 1):
#         print('-> PlotImgFunc: img is dark! skip it.')
#         return 'skipIt'
    
#     else:


    img_format = ".jpg"

    plt.gray() # This will show the images in grayscale as default

    plt.figure(figsize=(20,20))  # set the figure size

    plt.imshow(in_arr_2d, cmap='gray', vmin=0 , vmax=in_arr_2d.max())
    plt.show() # to show the img inline here

    if (save_mode==True):

        out_img = img_label + img_format
        out_img_fullpath = os.path.join(img_dir, out_img)
        print("-> saving output img as: %s" %out_img_fullpath)
        pltimg.imsave(out_img_fullpath, in_arr_2d)
        # plt.savefig(out_img)
        del plt
        return out_img_fullpath

    else:
        print('-> save mode is False, wo we onlu show ig here.')
        return 'onlyShowImg'

## set paths and check files

In [3]:
hdf_dir = '/Volumes/Ehsanm_DRI/research/MISR/hdf_files/ellipsoid_apr2013_day1to16_p1to233_b1to40/an_rad_images'
hdf_file = "MISR_AM1_GRP_ELLIPSOID_GM_P174_O070759_AN_F03_0024.hdf"

hdf_file_fullpath = os.path.join(hdf_dir, hdf_file) # platform independent
print(os.path.isfile(hdf_file_fullpath))

True


## some stats

In [4]:
hdf_obj = Mtk.MtkFile(hdf_file_fullpath)
print(type(hdf_obj))
total_block_range = hdf_obj.block
print(total_block_range)

<type 'MisrToolkit.MtkFile'>
(9, 152)


## process one image - Q- how read one block of data?

In [5]:
path = 174
block = 50
misr_res_meter = 275

In [6]:
my_block = Mtk.MtkRegion(path, block, block)
rad_arr = hdf_obj.grid('RedBand').field('Red Radiance/RDQI').read(my_block).data()
print(type(rad_arr))

<type 'numpy.ndarray'>


## Q- later how change array to object? or in memory?

In [7]:
# inDS = np.array(rad_arr, dtype=np.object)
# print(type(inDS))

## show image

In [8]:
img_label = 'path_'+str(path)+'_b'+str(block)
img_dir = hdf_dir
inDS = arr2img_func(rad_arr, img_label, img_dir)

-> save mode: True


<Figure size 640x480 with 0 Axes>

<Figure size 2000x2000 with 1 Axes>

-> saving output img as: /Volumes/Ehsanm_DRI/research/MISR/hdf_files/ellipsoid_apr2013_day1to16_p1to233_b1to40/an_rad_images/path_174_b50.jpg


In [9]:
raster_img_dir = hdf_dir
raster_fname = 'img2raster_'+'b'+str(block)+'_translated.tif'
raster_translated_fullpath = os.path.join(raster_img_dir, raster_fname)
print(raster_translated_fullpath)

/Volumes/Ehsanm_DRI/research/MISR/hdf_files/ellipsoid_apr2013_day1to16_p1to233_b1to40/an_rad_images/img2raster_b50_translated.tif


In [10]:
print("-> transfer img frame -> latlon frame for path_num: %d, block_num: %s" %(path, block))
#~ make 2 lists for img and geographic frames, then analyze long. to see if all are neg. or all are pos.; if all are the same then pass, else: change <-> to <+>
pixel_rowcol_list = []  # (row, column)
geographic_latlon_list = []


for jrow in [0, 200, 400, 511]:
    for icol in [0, 300, 600, 1000, 1300, 1700, 2047]:

        #~ process each col of each row
#             print("\n-> processing img coords: (%d, %d)" %(jrow, icol))

        #~ use MTK function to map from img frame to geographic frame
        pixel_latlon_tuple = Mtk.bls_to_latlon(path, misr_res_meter, block, jrow, icol) # struct=(lat, lon)?
        print(pixel_latlon_tuple)  # print every tuple of transfered latlon

        pixel_rowcol_list.append([jrow, icol])
        geographic_latlon_list.append(pixel_latlon_tuple)

print("-> total GCPs from img frame: %d" %len(pixel_rowcol_list))    
# print(pixel_rowcol_list)
# print(geographic_latlon_list)  # print to check all latlon list


#~ check all elements of long. to have same sign, either +/-
# all(iterable); iterable==list or anything that we can iterate on; Return True if all elements of the iterable are true
if (any( [LatLon_tuple[1] < 0  for  LatLon_tuple in geographic_latlon_list] )):   # if any Lon in geographic_latlon_list is neg. we change that to pos.
    ''' we enter this part if all or any pixels are on West hemisphere (have neg. long.) in img block, and we change them to pos. and the range will be [0, +360] '''

    print("-> note-1: found neg. lon. in geographic_latlon_list! will update neg. to pos. long.")

    gcp_list = []  # a list of ground control points

    for index, element in enumerate(pixel_rowcol_list):    # index == each GCP ; element order == [row, col]

        #~ for each point we add GCPs to a list
        gcp_list.append(gdal.GCP())  # initialize GCP dataStruct for each coordinate point

        #~ X,Y in img frame
        gcp_list[index].GCPLine  = pixel_rowcol_list[index][0]       # y == row
        gcp_list[index].GCPPixel = pixel_rowcol_list[index][1]       # x == column == pixel


        #~ chnage "-" Lon to "+" w.r.t CENTER_LONG = +180
        #~ print("-> updating neg. lon. to positive")

        #~ do this section for blocks that pass AntiMeridian, and we change all neg. long. to pos. long. that will span [0, 360] range 

        #~ 1st we add lat to list
        gcp_list[index].GCPY = geographic_latlon_list[index][0]  # lat=northing 

        #~ 2nd check for long. note; this section is as a result of any() meaning that we have 2 scenarios: some long. are neg./pos. or all are neg.
        pixel_long = geographic_latlon_list[index][1]

        if ( pixel_long < 0 ):  # if we find neg. long., we change it to pos. long. and will be in range [0, +360], especially case is all in neg. side of A.M.
            print("-> found neg. pixel long. in the crossing block: %f" % pixel_long)

            updated_lon_passed_AntiMerid = (360.0 + pixel_long )  # this long. is changed to pos. and will be in range [0, +360]
            print("=================> update: neg.long. updated to: %f" %updated_lon_passed_AntiMerid)

            #~ update neg. long. to pos. long.
            gcp_list[index].GCPX = updated_lon_passed_AntiMerid

        else:  # here we collect some pos. long. among neg. long. in the case block is crossing A.M. line

            # print("-> found pos. long. among neg. longs. in the crossing block: %f" %geographic_latlon_list[index][1])

            #~ no need to update Lon. since it is pos. Lon.
            gcp_list[index].GCPX = geographic_latlon_list[index][1]



else:
    print("-> note-2: all long. are on Eastern hemisphere (positive) in the img block!")

    gcp_list = []  # a list of ground control points

    for index, element in enumerate(pixel_rowcol_list):    # index == each GCP ; element order == [row, col]

        #~ for each point we add GCPs to a list
        gcp_list.append(gdal.GCP())  # initialize GCP dataStruct for each coordinate point

        #~ X,Y in img frame
        gcp_list[index].GCPLine  = pixel_rowcol_list[index][0]       # y == row
        gcp_list[index].GCPPixel = pixel_rowcol_list[index][1]       # x == column == pixel

        #~ add lat&lon
        gcp_list[index].GCPY = geographic_latlon_list[index][0]  # lat=northing 
        gcp_list[index].GCPX = geographic_latlon_list[index][1]  # lon=easting

-> transfer img frame -> latlon frame for path_num: 174, block_num: 50
(51.50172788126263, 37.98436790324279)
(51.39711465262676, 39.15842266023402)
(51.28078821388226, 40.32731202367263)
(51.10765745357027, 41.87652566772108)
(50.96445541237677, 43.03053877376678)
(50.75599599909551, 44.557408026654684)
(50.55921476448539, 45.869957345710525)
(51.01219486604948, 37.87996350155054)
(50.90843774784603, 39.041875198957)
(50.79316512692072, 40.198807962961105)
(50.62174066247859, 41.732412766081296)
(50.48003876411503, 42.87499959785881)
(50.273864336014654, 44.387048660346885)
(50.07932008653364, 45.687158208848416)
(50.522537648507125, 37.77744442429328)
(50.41961374118161, 38.92753971078439)
(50.30536822414074, 40.072833498142984)
(50.13560881215884, 41.59123696444536)
(49.99537216540821, 42.72268983291787)
(49.791431342611524, 44.22028899923788)
(49.59907540773796, 45.50826073426016)
(50.25072514075242, 37.72133488536304)
(50.14825413799848, 38.86500887406348)
(50.03456753908825, 40.0

In [11]:
translate_ds = gdal.Translate(
                                raster_translated_fullpath,
                                inDS,  
                                format = 'GTiff',
                                outputType = gdal.GDT_Float32, # maybe here chnage dtype to make it smaller img????
                                GCPs = gcp_list
                                ) 

In [12]:
print(os.path.isfile(raster_translated_fullpath))
print(raster_translated_fullpath)

True
/Volumes/Ehsanm_DRI/research/MISR/hdf_files/ellipsoid_apr2013_day1to16_p1to233_b1to40/an_rad_images/img2raster_b50_translated.tif


In [13]:
#~ Properly close the datasets to flush to disk
translate_ds = None
inDS = None

In [14]:
total_gcps = len(pixel_rowcol_list)

img_tag = 'test_warped'
warped_img = img_tag+"_reprojected_"+str(total_gcps)+"GCPs"+".tif"
outDS_warped = os.path.join(hdf_dir, warped_img)

## gdalwarp to apply transformation

In [15]:
from subprocess import call

# print(os.path.isfile(out_translated_fullpath))
# print(os.path.isfile(out_warped))

print('-> to cmdLine... \n')

cmd = 'gdalwarp\
    -r bilinear\
    -s_srs "EPSG:4326"\
    -t_srs "EPSG:4326"\
    -overwrite\
    -tps\
    -tr 0.001 0.001 \
    -co TILED=YES\
    --config CENTER_LONG +180\
    -srcnodata 0\
    -dstnodata 0 ' + raster_translated_fullpath + ' ' + outDS_warped  #  -tr 0.001 0.001\
    


#     print(cmd)

#~ note: shell=true to execute the line as full command including arguments
return_code = call(cmd, shell=True)  # subprocess.call() is better/safer that os.system() ==> finally used os.system() cos call returned error! correct return code?      


-> to cmdLine... 



## find hdf files based on pattern

In [16]:
# hdf_file_pattern = "MISR_AM1_GRP_ELLIPSOID_GM_P117_O088012_AN_F03_0024.hdf" #'MISR_AM1_GRP_ELLIPSOID_GM_P*'
# hdf_filelist = glob.glob(os.path.join(hdf_dir, hdf_file_pattern))
# print(hdf_filelist)

In [17]:
# print('-> number of MISR hdf files found: %d' % len(hdf_filelist))

## print numBlocks in a hdf file

In [18]:
# for hdf_file in hdf_filelist:
#     file_obj = Mtk.MtkFile(hdf_file)
#     total_block_range = file_obj.block
#     print('-> no. of blocks: (%d,%d)' % total_block_range)

## process one file at a time

In [19]:
# file_obj = Mtk.MtkFile(hdffile_fullpath)
# total_block_range = file_obj.block
# print('-> no. of blocks: (%d,%d)' % total_block_range)

## write numBlocks to file

In [20]:
# result_dir = home_dir
# out_filename = 'list_of_blockNum.txt'
# output_file = os.path.join(result_dir, out_filename)

# with open (output_file, 'w') as temp_write_obj:

#     for hdf_file in hdf_filelist:
#         file_obj = Mtk.MtkFile(hdf_file)
#         block_range = file_obj.block
#         # print('-> no. of blocks: (%d,%d)' % file_obj.block)
#         # print(block_range)
#         temp_write_obj.write(str(block_range)+'\n')		# write each string into output file


# print('-> WROTE SUCCESSFULLY!')

## define img directory

In [21]:
# img_dir = "/Users/ehsanmos/Documents/RnD/Ehsan_lab_MISR/"
# img_dir_label = 'img_dir.nosync'
# img_dir_fullpath = os.path.join(img_dir, img_dir_label)
# # print(img_dir_fp)
# print(os.path.isdir(img_dir_fullpath))

## img writter function

In [22]:
# def img_writer1(img_dir, img_name, img_arr):
#     '''writes images to img directory'''
    
#     #~ check if img dir exists?
#     if (cv2.imwrite(os.path.join(img_dir, img_name), img_arr) == True):
#         print("-> directory exists, writing to: %s" %img_dir)
#         print('-> writing img: %s' % img_name)
#     else:
#         print('-> ERROR writing img!')
#     return 0

## main() processing 

In [23]:
# grid_list = ['RedBand', 'GreenBand', 'BlueBand']
# field_list = ['Red Radiance', 'Green Radiance', 'Blue Radiance']
# # field_list = ['Red Radiance/RDQI', 'Green Radiance/RDQI', 'Blue Radiance/RDQI']

# # path = 100
# cam = 'AN'
# img_mode = 'rgb' #'rgb' or 'red_band'
# field_mode = 'noRDQI' # 'no_RDQI'

# file_obj = Mtk.MtkFile(hdffile_fullpath)

# for block_num in range(total_block_range[0], total_block_range[1], 1):
    
#     print('-> processing block: %d' % block_num)
    
#     #~ Q- how extract a block of data from hdf file??????
#     region_obj = Mtk.MtkRegion(path, block_num, block_num)  # maybe here?
    
    
#     if img_mode == 'red_band':
        
#         band_not_uniformed = file_obj.grid(grid_list[0]).field(field_list[0]).read(region_obj).data()   # read a band
#         band = band_not_uniformed
#         #print(type(band))
#         #print(type(band))
#         #print(band.ndim)
#         #print('-> array shape: (%d,%d)' % band.shape)
#         print('-> array (block, min, max): (%d, %d, %d)' %(block_num, band.min(), band.max()))
#         img_name = 'img_'+img_mode+'_AN_'+'path'+str(path)+'_block'+str(block_num)+'_'+field_mode+'.png'  #'.jpeg' #'.' #'  # or 
#         img_writer1(img_dir, img_name, band)
#         #del region_obj

            
#     if img_mode == 'rgb':
        
#         red =   file_obj.grid(grid_list[0]).field(field_list[0]).read(region_obj).data()
#         green = file_obj.grid(grid_list[1]).field(field_list[1]).read(region_obj).data()
#         blue =  file_obj.grid(grid_list[2]).field(field_list[2]).read(region_obj).data()
#         # field_data = field_buffer.read(region_obj).data()
#         factor = 1.0
#         rgb_arr_uint8 = (np.dstack((red,green,blue))*factor).astype(np.uint8) # stacks 3 arrays of rgb and changes the dtype to unsigned-int-8
#         #print(type(rgb_array_uint8))
#         print('-> array (block, min, max): (%d, %d, %d)' %(block_num, rgb_arr_uint8.min(), rgb_arr_uint8.max()))
#         img_name = 'img_RGB_AN_'+'path'+str(path)+'_block'+str(block_num)+'_'+field_mode+'.png' #'.jpeg'  # or png
#         img_writer1(img_dir_fullpath, img_name, rgb_arr_uint8)
        
# #         img = Image.fromarray(rgb_array_uint8)
# # #         img.save(os.path.join(img_dir, img_name))
        
   
        
        


In [24]:
print('-> SUCCESS END!')

-> SUCCESS END!


In [25]:
# print(type(field_data))
# print(field_data.ndim)
# print(field_data.shape)
# print(field_data.min())
# print(field_data.max())

In [26]:
# if (cv2.imwrite(os.path.join(img_dir, img_name), field_data) == True):
#     print('-> IMG WAS WRITTEN!')